In [1]:
#conda install -c plotly plotly=4.0.0

In [74]:
import plotly.graph_objects as go

import pandas as pd

In [82]:
#State Coords
# File to Load 
state_coords_to_load = "input/state_cords.csv"

# Read the firearms data
coord_data = pd.read_csv(state_coords_to_load)
coord_data.head()

,state,latitude,longitude,code
0,Alabama,32.806671,-86.791130,AL
1,Alaska,61.370716,-152.404419,AK
2,Arizona,33.729759,-111.431221,AZ
3,Arkansas,34.969704,-92.373123,AR
4,California,36.116203,-119.681564,CA


In [85]:
# File to Load 
fireams_data_to_load = "input/nics-firearm-background-checks.csv"

# Read the firearms data
firearms_data = pd.read_csv(fireams_data_to_load)

#Sort 2017
firearms_2017 = firearms_data[firearms_data['month'].str.contains('2017')]
registration_sum = firearms_2017.groupby(['state'])['totals'].sum()
registration_sum = registration_sum.reset_index()
#totals_df = pd.DataFrame([totals_sum]).T

state_handguns_sum = firearms_2017.groupby(['state'])['handgun'].sum()
state_handguns_sum = state_handguns_sum.reset_index()
#handgun_df = pd.DataFrame([state_handguns_sum]).T
#handgun_df['state'] = handgun_df.index
#handgun_df.head()

#sum longguns by state
state_longguns_sum = firearms_2017.groupby(['state'])['long_gun'].sum()
state_longguns_sum = state_longguns_sum.reset_index()

#longgun_df = pd.DataFrame([state_longguns_sum]).T
#longgun_df['state'] = longgun_df.index
#longgun_df.head()

data_pair = pd.merge(registration_sum, state_handguns_sum, how="left", on=["state", "state"])
totals_df = pd.merge(data_pair, state_longguns_sum, how="left", on=["state", "state"])

totals_df.head()

,state,totals,handgun,long_gun
0,Alabama,477345,97751.0,86210.0
1,Alaska,80839,34556.0,32648.0
2,Arizona,384930,153522.0,99248.0
3,Arkansas,237629,72100.0,76765.0
4,California,1570110,512465.0,318133.0


In [86]:
#World Atlas.COM
# File to Load 
state_population_to_load = "input/state_populations.csv"

# Read the firearms data
population_data = pd.read_csv(state_population_to_load)
population_data.head()

,pop_rank,state,population
0,1,California,"39,536,653"
1,2,Texas,"28,304,596"
2,3,Florida,"20,984,400"
3,4,New York,"19,849,399"
4,5,Pennsylvania,"12,805,537"


In [87]:
# Combine the data into a single dataset with a left merge on "state"
data_pair = pd.merge(totals_df, population_data, how="left", on=["state", "state"])
test_df = pd.merge(data_pair, coord_data, how="left", on=["state", "state"])
test_df.head()

,state,totals,handgun,long_gun,pop_rank,population,latitude,longitude,code
0,Alabama,477345,97751.0,86210.0,24.0,"4,874,747",32.806671,-86.791130,AL
1,Alaska,80839,34556.0,32648.0,48.0,"739,795",61.370716,-152.404419,AK
2,Arizona,384930,153522.0,99248.0,14.0,"7,016,270",33.729759,-111.431221,AZ
3,Arkansas,237629,72100.0,76765.0,32.0,"3,004,279",34.969704,-92.373123,AR
4,California,1570110,512465.0,318133.0,1.0,"39,536,653",36.116203,-119.681564,CA


In [89]:
for col in test_df.columns:
    test_df[col] = test_df[col].astype(str)

test_df['text'] = test_df['state'] + '<br>' + \
    'Total Registrations ' + test_df['totals'] + '<br>' + \
    'Handgun Registrations ' + test_df['handgun'] + '<br>' + \
    'Long Gun Registrations ' + test_df['long_gun'] + '<br>' + \
    'Population ' + test_df['population'] + '<br>' + \
    'Populations Rank ' + test_df['pop_rank']
    
fig = go.Figure(data=go.Choropleth(
    locations=test_df['code'],
    z=test_df['totals'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=test_df['text'], # hover text
    marker_line_color='black', # line markers between states
    colorbar_title="Millions of Registrations"
))

fig.update_layout(
    title_text='2017 Firearm Transactions Per State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()



In [43]:
df.head()

,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton,text
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61,Alabama<br>Beef 34.4 Dairy 4.06<br>Fruits 25.1...
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.0,0.6,1.0,1.56,0.0,0.0,0.0,Alaska<br>Beef 0.2 Dairy 0.19<br>Fruits 0.0 Ve...
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95,Arizona<br>Beef 71.3 Dairy 105.48<br>Fruits 60...
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44,Arkansas<br>Beef 53.2 Dairy 3.53<br>Fruits 6.8...
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.4,803.2,1303.5,2106.79,34.6,249.3,1064.95,California<br>Beef 228.7 Dairy 929.95<br>Fruit...


In [42]:
#import plotly.graph_objects as go

#import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in df.columns:
    df[col] = df[col].astype(str)

df['text'] = df['state'] + '<br>' + \
    'Beef ' + df['beef'] + ' Dairy ' + df['dairy'] + '<br>' + \
    'Fruits ' + df['total fruits'] + ' Veggies ' + df['total veggies'] + '<br>' + \
    'Wheat ' + df['wheat'] + ' Corn ' + df['corn']

fig = go.Figure(data=go.Choropleth(
    locations=df['code'],
    z=df['total exports'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    autocolorscale=False,
    text=df['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title="Millions USD"
))

fig.update_layout(
    title_text='2011 US Agriculture Exports by State<br>(Hover for breakdown)',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

In [16]:


#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
#df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)
df = df['totals'] + '' + df['state'] + '' + 'Transactions: ' + df['cnt'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = df['longitude'],
        lat = df['latitude'],
        #text = df['text'],
        #mode = 'markers',
        marker_color = df['cnt'],
        ))

fig.update_layout(
        title = 'Total firearm transactions per state <br>(Hover over state)',
        geo_scope='usa',
    )
fig.show()

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U21') dtype('<U21') dtype('<U21')